In [9]:
import argparse
import os
import time
import shutil
import torch.multiprocessing as mp
import torch.nn.parallel
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.tensorboard import SummaryWriter
from models import VideoModel
from utils.transforms import *
from utils.opts import parser
import utils.CosineAnnealingLR as CosineAnnealingLR
import utils.datasets_video as datasets_video
from utils.dataset import VideoDataset
from datetime import datetime
import os
import numpy
import pickle as pkl
from torch.cuda import amp

In [12]:
model = VideoModel(num_class=174, num_segments=8,
                    base_model='bninception', consensus_type='avg', dropout=0.5,
                    gsf=True, gsf_ch_ratio=100,
                    target_transform=None)


    Initializing Video Model with backbone: bninception.
    Model Configurations:
                        GSF:                True
                        Channel ratio:      100
                        num_segments:       8
                        consensus_module:   avg
                        dropout_ratio:      0.5
            
No. of GSF modules = 10


In [13]:
# lead pretrained weights
checkpoint = torch.load("../something-v1_bninception_16frames.pth.tar", map_location=torch.device('mps'))

base_dict = {'.'.join(k.split('.')[1:]): v for k,v in list(checkpoint['model_state_dict'].items())}
model.load_state_dict(base_dict, strict=True)

<All keys matched successfully>

In [14]:
# freeze layers
for param in model.parameters():
    param.requires_grad = False

In [15]:
# get num features of last layer
num_ftrs = model.new_fc.in_features
num_ftrs

1024

In [16]:
# rewrite last layer to be trained
model.new_fc = nn.Linear(num_ftrs, 4)